In [ ]:
from PIL import Image
import imagehash
import os
import csv
import numpy as np
from os.path import exists
path='output.csv'
csv_file= exists(path)

if not csv_file:
    with open('output.csv','a',encoding='utf-8',errors='ignore',newline='') as f:
            writer = csv.writer(f,delimiter='|')
            writer.writerow(["id","first_images","images_to_match_with"])

def save_in_csv(wr): 
    with open('output.csv', 'a',encoding='utf-8',errors='ignore',newline='') as file:
        writer = csv.writer(file,delimiter='|')
        writer.writerow(wr)

class DuplicateRemover:
    def __init__(self,dirname,hash_size = 8):
        self.dirname = dirname
        self.hash_size = hash_size
        
    def find_duplicates(self):
        """
        Find and Delete Duplicates
        """
        a=[]
        fnames = os.listdir(self.dirname)
        hashes = {}
        duplicates = []
        print("Finding Duplicates Now!\n")
        for image in fnames:
            with Image.open(os.path.join(self.dirname,image)) as img:
                temp_hash = imagehash.average_hash(img, self.hash_size)
                if temp_hash in hashes:
                    print("Duplicate {} \nfound for Image {}!\n".format(image,hashes[temp_hash]))
                    duplicates.append(image)
#                     print(duplicates)
                    
                    a.append(image)
                else:
                    hashes[temp_hash] = image
                    
                    
        return a
            
                   

            
    def find_similar(self,location,similarity=80):
        fnames = os.listdir(self.dirname)
        threshold = 1 - similarity/100
        diff_limit = int(threshold*(self.hash_size**2))
        
        with Image.open(location) as img:
            hash1 = imagehash.average_hash(img, self.hash_size).hash
        
        print("Finding Similar Images to {} Now!\n".format(location))
        for image in fnames:
            with Image.open(os.path.join(self.dirname,image)) as img:
                hash2 = imagehash.average_hash(img, self.hash_size).hash
                
                if np.count_nonzero(hash1 != hash2) <= diff_limit:
                    print("{} image found {}% similar to {}".format(image,similarity,location))
                    
                    
with open('input.csv') as file_obj:
      
    # Skips the heading
    # Using next() method
    heading = next(file_obj)
      
    # Create reader object by passing the file 
    # object to reader method
    reader_obj = csv.reader(file_obj)
      
    # Iterate over each row in the csv file 
    # using reader object
    for row in reader_obj:
        l1=row[1].split("~")
        l2=row[2].split("~")
#         print(l1)
#         print(l2)
        a=[]
        dict1={}
        import requests # request img from web
        import shutil # save img locally
        for i in l1[0:]:
            url = i #getting image url
            file_name = 'main.jpeg'#location of file where you will save downloaded image file

            res = requests.get(url, stream = True)

            if res.status_code == 200:
                with open(file_name,'wb') as f:
                    shutil.copyfileobj(res.raw, f)
                print('Image sucessfully Downloaded: ',file_name)
                #print(i);
            else:
                print('Image Couldn\'t be retrieved')
            dirname = "/all_images/"#directory where both images to be mathced will be saved

            for j , img in enumerate(l2):
                dict1['image'+str(i)+'.jpeg']=str(img)

                url = img #prompt user for img url
                file_name = 'findimage.jpeg'#directory for saving another image

                res = requests.get(url, stream = True)

                if res.status_code == 200:
                    with open(file_name,'wb') as f:
                        shutil.copyfileobj(res.raw, f)
                    print('Image sucessfully Downloaded: ',file_name)
                    print(img)
                else:

                    print('Image Couldn\'t be retrieved')
                dr = DuplicateRemover(dirname) #find duplicates
                b=dr.find_duplicates()
                if len(b)>0:
                    print("---------",i,"---",img)
                    w=[row[0],i,img]
                    save_in_csv(w)

        

         
